# NetworkX

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Ouverture

In [ ]:
symmetric = nx.read_edgelist('edges_list.txt',
                             comments="node",
                             create_using=nx.Graph(),nodetype=int)
asymmetric = nx.read_edgelist('edges_list.txt',
                             comments="node",
                             create_using=nx.DiGraph(),nodetype=int)
G_undirected = nx.DiGraph.to_undirected(asymmetric)
G = asymmetric

In [ ]:
### Alternative way to read the edgelist (1st step)
df = pd.read_csv('edge_list.txt', sep='\t')
G_from_df = nx.from_pandas_edgelist(df, 'node1', 'node2', create_using=nx.DiGraph())

## With attributes
nodes_with_attributes = pd.read_table("node_attributes.txt", sep='\t')
nodes_with_attributes = pd.read_table("node_attributes.txt", sep='\t')
nodes_with_attributesData = nodes_with_attributes.set_index('Node').to_dict('index').items()
G.add_nodes_from(nodes_with_attributesData)

Adding note attributes can create isolates that do not appear in the edge list (by definition) :

In [ ]:
# get nodes index of isolates

list(nx.isolates(G)), list(nx.isolates(G_undirected))

In [ ]:
########### Simple graph generators ###########

# complete graph
G1 = nx.complete_graph (10)
# chain
G2 = nx.path_graph (10)


In [ ]:
# remove isolates
G_without_isolates = G.copy() # we make a copy because we will need the initial graph afterwards
G_without_isolates.remove_nodes_from(list(nx.isolates(G)))

## Simple visualisation

In [ ]:
nx.draw(G)

nx.draw(G, with_labels=False, node_size=30, node_color="red", edge_color = "gray",
        pos=nx.fruchterman_reingold_layout(G))

# to display proprieties

### use different colors to represent attributes, eg Office
# Office has 3 values only (categorical variable)

# create empty list for node colors
node_color = []

# for each node in the graph
for node in G.nodes(data=True):

    # if the node has the attribute Office 1
    if node[1]['Office']==1:
        node_color.append('blue')
        
    # if the node has the attribute Office 2
    elif node[1]['Office']==2:
        node_color.append('red')

    # if the node has the attribute office 3
    elif node[1]['Office']==3:
        node_color.append('yellow')

    # draw graph with node attribute color
nx.draw(G, with_labels=False, node_size=25, node_color=node_color, edge_color = "gray")


## properties

### Simple ones

In [ ]:
# Simple properties:  number of nodes 
G.number_of_nodes() # or: len(G)
G.nodes()

In [ ]:
# Simple properties: number of edges
G.number_of_edges ()
G.edges()

In [ ]:
# Degree ( = number of edges incident to each node)
G.degree ()
# Degree of a specific node ( = the number of edges incident to it)
G.degree (1)

### stats structures

In [ ]:
##### Isolates (components consisting in 1 node) ### 

list(nx.isolates(G))

##### Dyads: reciprocity (only interesting in digraph; not relevant in undirected graphs)

nx.reciprocity(G)

##### Triads: global transitivity
print(nx.transitivity(G)) # this is the global clustering coefficient

##### Triads: local transitivity
print(nx.clustering(G))  # measure is by node (local clustering coefficient)
## take the average:
print(nx.average_clustering(G)) 

In [ ]:
##### Density
nx.density(G)

### /!\ find cliques /!\

# /!\ the networkx algorithm is only for undirected graphs
list(nx.find_cliques(G_undirected))

################## Basic connectivity measures

# is the graph connected?
print(nx.is_strongly_connected(G)) # "strongly" because it is a digraph
# with an undirected graph, nx.is_connected(G)

# if "strongly" is false, try with "weakly":
print(nx.is_weakly_connected(G))

print(nx.is_connected(G)) # false if there are isolates

## diameter

# because graph is not strongly connected (infinite distances)
# we need to take its undirected representation
# ELadviceUG = nx.DiGraph.to_undirected(ELadviceG)
# /!\ the networkx algorithm is only for undirected graphs
print(nx.diameter(G_undirected))

## average shortest path length
# because graph is not strongly connected (infinite distances)
# we need to take its undirected representation
# ELadviceUG = nx.DiGraph.to_undirected(ELadviceG)
# /!\ the networkx algorithm is only for undirected graphs
nx.average_shortest_path_length(G_undirected)

In [ ]:
################# Centrality

# degree centrality (without considering directed edges)
print(nx.degree_centrality(G))

# indegree centrality (incoming ties)
print(nx.in_degree_centrality(G))

# indegree centrality (outgoing ties)
print(nx.out_degree_centrality(G))

# Eigenvector centrality

nx.eigenvector_centrality(G_undirected)

# This measure works only with undirected graphs.
# For directed graphs, networkx takes the “left” eigenvector centrality
# which corresponds to the in-edges in the graph.
# For out-edges eigenvector centrality first reverse the graph with G.reverse()
# (it is the transpose of the adjacency matrix)

# Closeness centrality
nx.closeness_centrality(G)

# The closeness centrality uses inward distance to a node, not outward.
# If you want to use outword distances apply the function to G.reverse()

# Betweenness centrality
nx.betweenness_centrality(G)

# Betweenness values are normalized by 2/((n-1)(n-2)) for graphs,
# and 1/((n-1)(n-2)) for directed graphs, where n is the number of nodes 

#### Let's briefly look at degree distributions
indegrees = [val for (node, val) in G.in_degree()]
outdegrees = [val for (node, val) in G.out_degree()]

In = np.array(indegrees)
Out = np.array(outdegrees)

plt.hist([In, Out], bins=5, label=['Indegrees', 'Outdegrees'])
plt.legend(loc='upper right')
plt.show()

In [ ]:
# first create a function
def blau(df, col):
    return (1- ((df[col].value_counts() / df[col].count()) ** 2).sum())

# then apply it to the 'alters' table of ego 28, for example with attribute 'alter.age.cat' (= age category)
blau(G, 'categorial_var')

# We can now calculate the IQV

blau(G, 'categorial_var')*7/6

# Herfindahl-Hirschman index (HHI)
# equal to 1 - Blau

1 - blau(G, 'categorial_var')

## Export

In [ ]:
###### For better visualization
## export graph in Gephi format

nx.write_gexf(G, 'G.gexf')